<a href="https://colab.research.google.com/github/msfasha/Arabic-Deep-Learning-OCR/blob/master/Arabic_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usage:<br>
1- goto the dataset repository:<br>
https://drive.google.com/drive/u/2/folders/1mRefmN4Yzy60Uh7z3B6cllyyOXaxQrgg
 
and select one of the datasets e.g. 1_nice_60000_rows

2- download the related files, for example:<br>
a- 1_nice_60000_rows.bin<br>
and<br>
b- 1_nice_60000_rows.txt<br><br>

2- upload the dataset files you downloaded from the dataset repository into your Google Colab drive.
if you uploaded the files into Colab drive, they will be deleted once the session is over. Hence, you can upload the downloaded dataset files into your own google drive and mount your google drive in Colab.

4- Set the path for the dataset files in config.py below i.e. whether in Colab drive or your own google drive e.g. /content/drive/MyDrive/

5- initially, we need to train the model, so we set the 
**OPERATION_TYPE = OperationType.Training** value in config.py below to Training.

6- If your are using the dataset for the first time, make sure to empty your output folder from any previously generated files (dataset split files and model files...etc).

7- Set the experiment name via the experiment_name constant below (if required)

8- Run the training process, the training process will stop after 5 epoches with no improvements, you can change this value in config.py below.

9- After finishing the training process, you can test the model on the testing dataset by setting the **OPERATION_TYPE = OperationType.Testing**

10- Finally, you can make inference on single images by setting **OPERATION_TYPE = OperationType.Infer**
To make inference, you need to place the required word image file in the designated folder defined by the INDIVIDUAL_TEST_IMAGE_PATH constant value below.
You also need to set the filename for the image you are trying to infer in the fnInfer variable below e.g. 0.png.

You can find sample files in the dataset repository in Google drive as well as the dataset folder in GitHub (under the sample_dataset_imgages folder)


Config.py

In [ ]:
%cd /content/drive/MyDrive/ArabicMultiFontsDataset/
%ls

In [ ]:
# The location/path of the uploaded dataset files (after downloading them from the dataset repository)
# Make sure to mount your Google drive in Colab
BASE_PATH = "/content/drive/MyDrive/ArabicMultiFontsDataset/"

# The name of the dataset files i.e. the binary file and the labels file.
BASE_FILENAME = "1_nice_60000_rows"

In [ ]:
%tensorflow_version 1.x

from __future__ import division
from __future__ import print_function
import os
from enum import Enum
import numpy as np
import argparse
from datetime import datetime
import time
import cv2
import random
import sys
from shutil import Error
import tensorflow as tf
import editdistance

class OperationType(Enum):
    Training = 1
    Validation = 2
    Testing = 3
    Infer = 4


class DecoderType:
    BestPath = 0
    BeamSearch = 1
    WordBeamSearch = 2


# Experiment name, to be saved in the audit log.
EXPERIMENT_NAME = "Test Drive Training Process"

# The type of the run session, depending on this type, designated datasets will be loaded.
# Set this value to training to run a training process using the testing dataset.
# Set this value to testing to run a testing process using the testing dataset.
# Set this value to infer, to make an inference for a single word image file, make sure
# to place the required word image file in the directory defined by the INDIVIDUAL_TEST_IMAGE_PATH below, and
# make sure to set the fnInfer to the name of the image file your are trying to infer
OPERATION_TYPE = OperationType.Training

DECODER_TYPE = DecoderType.BestPath

# Use this value to regenerate the training/validation/test datasets, as well as
# the other support files. Usually this is needed when we start the training process
# It is not needed during the Testing process so we set it to true
# in order to regenerate all the required files, we have to delete to old ones train/validate/test and delete
# and then its value to true. After running the app, the files are generated and we can set it back to false unless
# we need to generate a new set of data i.e. train/validate/test
REGENERATE_CHARLIST_AND_CORPUS = True


#You can modify these folder settings according to your preference

#Set the path where the train, validate,test datasets are saved
DATA_PATH = BASE_PATH 
#set the path where to save the generated tensorflow model
MODEL_PATH = BASE_PATH
#set the path for the autogenerated files
OUTPUT_PATH = BASE_PATH

#Set the path of the single image files that you want to recognize
INDIVIDUAL_TEST_IMAGE_PATH = DATA_PATH

BASE_IMAGES_FILE = DATA_PATH + BASE_FILENAME + ".bin"
BASE_LABELS_FILE = DATA_PATH + BASE_FILENAME + ".txt"
TRAINING_LABELS_FILE = DATA_PATH + "TRAINING_DATA_" + BASE_FILENAME + ".txt"
VALIDATION_LABELS_FILE = DATA_PATH + \
    "VALIDATION_DATA_" + BASE_FILENAME + ".txt"
TESTING_LABELS_FILE = DATA_PATH + "TESTING_DATA_" + BASE_FILENAME + ".txt"
fnCharList = OUTPUT_PATH + 'charList.txt'
fnResult = OUTPUT_PATH + 'result.txt'

# define the name of the word image file you want to test/infer
# for example, you can select 0.png file from as a sample
#https://github.com/msfasha/Arabic-Deep-Learning-OCR/tree/master/dataset/sample_files
fnInfer = INDIVIDUAL_TEST_IMAGE_PATH + "sample_files/" + "0.png"


fnCorpus = OUTPUT_PATH + 'corpus.txt'
fnwordCharList = OUTPUT_PATH + 'wordCharList.txt'

# Number of batches for each epoch = SAMPLES_PER_EPOCH / BATCH_SIZE
TRAINING_SAMPLES_PER_EPOCH = 5000
BATCH_SIZE = 100
VALIDATIOIN_SAMPLES_PER_STEP = (int)(TRAINING_SAMPLES_PER_EPOCH * .2)


TRAINING_DATASET_SIZE = .9
# .5 of the remaining ==> (Total - TRAINING_DATASET_SIZE) / 2
VALIDATION_DATASET_SPLIT_SIZE = .5
# stop after no improvements for this number of epochs
MAXIMUM_NONIMPROVED_EPOCHS = 5
MAXIMUM_MODELS_TO_KEEP = 3  # usually only 1, the last one

#IMAGE_SIZE = (128, 32)
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 32
MAX_TEXT_LENGTH = 32
RESIZE_IMAGE = True
CONVERT_IMAGE_TO_MONOCHROME = False
MONOCHROME_BINARY_THRESHOLD = 127
AUGMENT_IMAGE = False

def auditLog(logStr):
    open(fnResult, 'a').write(logStr)


Model.py

In [ ]:
class Model:
    "minimalistic TF model for HTR"

    def __init__(self, decoderType = DecoderType.BestPath, mustRestore=False, dump=False):
        "init model: add CNN, RNN and CTC and initialize TF"
        self.dump = dump
        self.charList = open(fnCharList, encoding="utf-8").read()
        self.decoderType = decoderType
        self.mustRestore = mustRestore
        self.snapID = 0

        # Whether to use normalization over a batch or a population
        self.is_train = tf.placeholder(tf.bool, name='is_train')

        # input image batch
        self.inputImgs = tf.placeholder(tf.float32, shape=(
            None, IMAGE_WIDTH, IMAGE_HEIGHT))

        # setup CNN, RNN and CTC
        self.setup5LayersCNN()
        self.setupRNN()
        self.setupCTC()

        # setup optimizer to train NN
        self.batchesTrained = 0
        self.learningRate = tf.placeholder(tf.float32, shape=[])
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
            self.optimizer = tf.train.RMSPropOptimizer(
                self.learningRate).minimize(self.loss)

        self.auditModelDetails()
        # initialize TF
        (self.sess, self.saver) = self.setupTF()

    def auditModelDetails(self):
        total_parameters = 0
        saveString = "Model Details" + "\n"
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            saveString = saveString + "Shape:" + \
                str(shape) + " ,shape length:" + str(len(shape))
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            saveString = saveString + " , parameters: " + \
                str(variable_parameters) + "\n"
            total_parameters += variable_parameters

        saveString = saveString + "Total Parameters: " + \
            str(total_parameters) + "\n\n"

        print(saveString)
        auditLog(saveString)

    def setup5LayersCNN(self):
        "create CNN layers and return output of these layers"
        cnnIn4d = tf.expand_dims(input=self.inputImgs, axis=3)
        pool = cnnIn4d  # input to first CNN layer

        self.kernel1 = tf.Variable(
            tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
        self.conv1 = tf.nn.conv2d(
            pool, self.kernel1, padding='SAME', strides=(1, 1, 1, 1))
        conv_norm = tf.layers.batch_normalization(
            self.conv1, training=self.is_train)
        self.relu1 = tf.nn.relu(conv_norm)
        self.pool1 = tf.nn.max_pool(
            self.relu1, (1, 2, 2, 1), (1, 2, 2, 1), 'VALID')

        kernel = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
        self.conv2 = tf.nn.conv2d(
            self.pool1, kernel, padding='SAME', strides=(1, 1, 1, 1))
        conv_norm = tf.layers.batch_normalization(
            self.conv2, training=self.is_train)
        relu = tf.nn.relu(conv_norm)
        pool = tf.nn.max_pool(relu, (1, 2, 2, 1), (1, 2, 2, 1), 'VALID')

        kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev=0.1))
        self.conv3 = tf.nn.conv2d(
            pool, kernel, padding='SAME', strides=(1, 1, 1, 1))
        conv_norm = tf.layers.batch_normalization(
            self.conv3, training=self.is_train)
        relu = tf.nn.relu(conv_norm)
        pool = tf.nn.max_pool(relu, (1, 1, 2, 1), (1, 1, 2, 1), 'VALID')

        kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], stddev=0.1))
        self.conv4 = tf.nn.conv2d(
            pool, kernel, padding='SAME', strides=(1, 1, 1, 1))
        conv_norm = tf.layers.batch_normalization(
            self.conv4, training=self.is_train)
        relu = tf.nn.relu(conv_norm)
        pool = tf.nn.max_pool(relu, (1, 1, 2, 1), (1, 1, 2, 1), 'VALID')

        kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], stddev=0.1))
        self.conv5 = tf.nn.conv2d(
            pool, kernel, padding='SAME', strides=(1, 1, 1, 1))
        conv_norm = tf.layers.batch_normalization(
            self.conv5, training=self.is_train)
        relu = tf.nn.relu(conv_norm)
        pool = tf.nn.max_pool(relu, (1, 1, 2, 1), (1, 1, 2, 1), 'VALID')

        self.cnnOut4d = pool

    def setupCNN7Layers(self):
        "create CNN layers and return output of these layers"
        cnnIn4d = tf.expand_dims(input=self.inputImgs, axis=3)

        kernel1 = tf.Variable(tf.truncated_normal([3, 3, 1, 64], stddev=0.1))
        conv1 = tf.nn.conv2d(
            cnnIn4d, kernel1, padding='SAME', strides=(1, 1, 1, 1))
        pool1 = tf.nn.max_pool(conv1, (1, 2, 2, 1), (1, 2, 2, 1), 'VALID')

        kernel2 = tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev=0.1))
        conv2 = tf.nn.conv2d(
            pool1, kernel2, padding='SAME', strides=(1, 1, 1, 1))
        pool2 = tf.nn.max_pool(conv2, (1, 2, 2, 1), (1, 2, 2, 1), 'VALID')

        kernel3 = tf.Variable(tf.truncated_normal(
            [3, 3, 128, 256], stddev=0.1))
        conv3 = tf.nn.conv2d(
            pool2, kernel3, padding='SAME', strides=(1, 1, 1, 1))

        kernel4 = tf.Variable(tf.truncated_normal(
            [3, 3, 256, 256], stddev=0.1))
        conv4 = tf.nn.conv2d(
            conv3, kernel4, padding='SAME', strides=(1, 1, 1, 1))
        pool3 = tf.nn.max_pool(conv4, (1, 1, 2, 1), (1, 1, 2, 1), 'VALID')

        kernel5 = tf.Variable(tf.truncated_normal(
            [3, 3, 256, 512], stddev=0.1))
        conv5 = tf.nn.conv2d(
            pool3, kernel5, padding='SAME', strides=(1, 1, 1, 1))
        batch_norm1 = tf.layers.batch_normalization(
            conv4, training=self.is_train)

        kernel6 = tf.Variable(tf.truncated_normal(
            [3, 3, 512, 512], stddev=0.1))
        conv6 = tf.nn.conv2d(batch_norm1, kernel6,
                             padding='SAME', strides=(1, 1, 1, 1))
        batch_norm2 = tf.layers.batch_normalization(
            conv6, training=self.is_train)
        pool4 = tf.nn.max_pool(batch_norm2, (1, 1, 2, 1),
                               (1, 1, 2, 1), 'VALID')

        kernel7 = tf.Variable(tf.truncated_normal(
            [2, 2, 512, 512], stddev=0.1))
        conv7 = tf.nn.conv2d(batch_norm1, kernel7,
                             padding='SAME', strides=(1, 1, 1, 1))

        self.cnnOut4d = conv7

    def setupRNN(self):
        "create RNN layers and return output of these layers"
        rnnIn3d = tf.squeeze(self.cnnOut4d, axis=[2])

        # basic cells which is used to build RNN
        numHidden = 256
        cells = [tf.contrib.rnn.LSTMCell(
            num_units=numHidden, state_is_tuple=True) for _ in range(2)]  # 2 layers

        # stack basic cells
        stacked = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

        # bidirectional RNN
        # BxTxF -> BxTx2H
        ((fw, bw), _) = tf.nn.bidirectional_dynamic_rnn(cell_fw=stacked, cell_bw=stacked, inputs=rnnIn3d,
                                                        dtype=rnnIn3d.dtype)

        # BxTxH + BxTxH -> BxTx2H -> BxTx1X2H
        concat = tf.expand_dims(tf.concat([fw, bw], 2), 2)

        # project output to chars (including blank): BxTx1x2H -> BxTx1xC -> BxTxC
        kernel = tf.Variable(tf.truncated_normal(
            [1, 1, numHidden * 2, len(self.charList) + 1], stddev=0.1))
        self.rnnOut3d = tf.squeeze(tf.nn.atrous_conv2d(
            value=concat, filters=kernel, rate=1, padding='SAME'), axis=[2])

    def setupCTC(self):
        "create CTC loss and decoder and return them"
        # BxTxC -> TxBxC
        self.ctcIn3dTBC = tf.transpose(self.rnnOut3d, [1, 0, 2])

        # ground truth text as sparse tensor
        self.gtTexts = tf.SparseTensor(tf.placeholder(tf.int64, shape=[None, 2]), tf.placeholder(tf.int32, [None]),
                                       tf.placeholder(tf.int64, [2]))

        # calc loss for batch
        self.seqLen = tf.placeholder(tf.int32, [None])
        self.loss = tf.reduce_mean(
            tf.nn.ctc_loss(labels=self.gtTexts, inputs=self.ctcIn3dTBC, sequence_length=self.seqLen,
                           ctc_merge_repeated=True))

        # calc loss for each element to compute label probability
        self.savedCtcInput = tf.placeholder(
            tf.float32, shape=[MAX_TEXT_LENGTH, None, len(self.charList) + 1])
        self.lossPerElement = tf.nn.ctc_loss(labels=self.gtTexts, inputs=self.savedCtcInput,
                                             sequence_length=self.seqLen, ctc_merge_repeated=True)

        # decoder: either best path decoding or beam search decoding
        if self.decoderType == DecoderType.BestPath:
            self.decoder = tf.nn.ctc_greedy_decoder(
                inputs=self.ctcIn3dTBC, sequence_length=self.seqLen)
        elif self.decoderType == DecoderType.BeamSearch:
            self.decoder = tf.nn.ctc_beam_search_decoder(inputs=self.ctcIn3dTBC, sequence_length=self.seqLen,
                                                         beam_width=50, merge_repeated=False)
        elif self.decoderType == DecoderType.WordBeamSearch:
            # import compiled word beam search operation (see https://github.com/githubharald/CTCWordBeamSearch)
            word_beam_search_module = tf.load_op_library('TFWordBeamSearch.so')

            # prepare information about language (dictionary, characters in dataset, characters forming words)
            chars = str().join(self.charList)
            wordChars = open(fnwordCharList).read().splitlines()[0]
            corpus = open(fnCorpus).read()

            # decode using the "Words" mode of word beam search
            self.decoder = word_beam_search_module.word_beam_search(tf.nn.softmax(self.ctcIn3dTBC, dim=2), 50, 'Words',
                                                                    0.0, corpus.encode(
                                                                        'utf8'), chars.encode('utf8'),
                                                                    wordChars.encode('utf8'))

    def setupTF(self):
        "initialize TF"
        print('Python: ' + sys.version)
        print('Tensorflow: ' + tf.__version__)

        sess = tf.Session()  # TF session

        # saver saves model to file
        saver = tf.train.Saver(max_to_keep=MAXIMUM_MODELS_TO_KEEP)
        modelDir = MODEL_PATH
        latestSnapshot = tf.train.latest_checkpoint(
            modelDir)  # is there a saved model?

        # if model must be restored (for inference), there must be a snapshot
        if self.mustRestore and not latestSnapshot:
            raise Exception('No saved model found in: ' + modelDir)

        # load saved model if available
        if latestSnapshot:
            print('Init with stored values from ' + latestSnapshot)
            saver.restore(sess, latestSnapshot)
        else:
            print('Init with new values')
            sess.run(tf.global_variables_initializer())

        return (sess, saver)

    def toSparse(self, texts):
        "put ground truth texts into sparse tensor for ctc_loss"
        indices = []
        values = []
        shape = [len(texts), 0]  # last entry must be max(labelList[i])

        # go over all texts
        for (batchElement, text) in enumerate(texts):
            # convert to string of label (i.e. class-ids)
            CharactersIndexesOflabels = [self.charList.index(c) for c in text]
            # sparse tensor must have size of max. label-string
            if len(CharactersIndexesOflabels) > shape[1]:
                shape[1] = len(CharactersIndexesOflabels)
            # put each label into sparse tensor
            for (i, label) in enumerate(CharactersIndexesOflabels):
                indices.append([batchElement, i])
                values.append(label)

        return (indices, values, shape)

    def decoderOutputToText(self, ctcOutput, batchSize):
        "extract texts from output of CTC decoder"

        # contains string of labels for each batch element
        encodedLabelStrs = [[] for i in range(batchSize)]

        # word beam search: label strings terminated by blank
        if self.decoderType == DecoderType.WordBeamSearch:
            blank = len(self.charList)
            for b in range(batchSize):
                for label in ctcOutput[b]:
                    if label == blank:
                        break
                    encodedLabelStrs[b].append(label)

        # TF decoders: label strings are contained in sparse tensor
        else:
            # ctc returns tuple, first element is SparseTensor
            decoded = ctcOutput[0][0]

            # go over all indices and save mapping: batch -> values
            idxDict = {b: [] for b in range(batchSize)}
            for (idx, idx2d) in enumerate(decoded.indices):
                label = decoded.values[idx]
                batchElement = idx2d[0]  # index according to [b,t]
                encodedLabelStrs[batchElement].append(label)

        # map labels to chars for all batch elements
        return [str().join([self.charList[c] for c in labelStr]) for labelStr in encodedLabelStrs]

    def trainBatch(self, batch):
        "feed a batch into the NN to train it"
        numBatchElements = len(batch.imgs)
        sparse = self.toSparse(batch.gtTexts)
        rate = 0.01 if self.batchesTrained < 10 else (
            0.001 if self.batchesTrained < 10000 else 0.0001)  # decay learning rate
        evalList = [self.optimizer, self.loss]
        feedDict = {self.inputImgs: batch.imgs, self.gtTexts: sparse,
                    self.seqLen: [MAX_TEXT_LENGTH] * numBatchElements, self.learningRate: rate,
                    self.is_train: True}

        (_, lossVal) = self.sess.run(evalList, feedDict)

        self.batchesTrained += 1
        return lossVal

    def dumpNNOutput(self, rnnOutput):
        "dump the output of the NN to CSV file(s)"
        dumpDir = '../dump/'
        if not os.path.isdir(dumpDir):
            os.mkdir(dumpDir)

        # iterate over all batch elements and create a CSV file for each one
        maxT, maxB, maxC = rnnOutput.shape
        for b in range(maxB):
            csv = ''
            for t in range(maxT):
                for c in range(maxC):
                    csv += str(rnnOutput[t, b, c]) + ';'
                csv += '\n'
            fn = dumpDir + 'rnnOutput_' + str(b) + '.csv'
            print('Write dump of NN to file: ' + fn)
            with open(fn, 'w') as f:
                f.write(csv)

    def inferBatch(self, batch, calcProbability=False, probabilityOfGT=False):
        "feed a batch into the NN to recognize the texts"

        # decode, optionally save RNN output
        numBatchElements = len(batch.imgs)
        evalRnnOutput = self.dump or calcProbability
        evalList = [self.decoder] + \
            ([self.ctcIn3dTBC] if evalRnnOutput else [])
        feedDict = {self.inputImgs: batch.imgs, self.seqLen: [MAX_TEXT_LENGTH] * numBatchElements,
                    self.is_train: False}

        evalRes = self.sess.run(evalList, feedDict)

        decoded = evalRes[0]
        texts = self.decoderOutputToText(decoded, numBatchElements)

        # feed RNN output and recognized text into CTC loss to compute labeling probability
        probs = None
        if calcProbability:
            sparse = self.toSparse(
                batch.gtTexts) if probabilityOfGT else self.toSparse(texts)
            ctcInput = evalRes[1]
            evalList = self.lossPerElement
            feedDict = {self.savedCtcInput: ctcInput, self.gtTexts: sparse,
                        self.seqLen: [MAX_TEXT_LENGTH] * numBatchElements, self.is_train: False}

            lossVals = self.sess.run(evalList, feedDict)

            probs = np.exp(-lossVals)

        # dump the output of the NN to CSV file(s)
        if self.dump:
            self.dumpNNOutput(evalRes[1])

        return (texts, probs)

    def save(self):
        "save model to file"
        self.snapID += 1
        self.saver.save(self.sess, MODEL_PATH +
                        EXPERIMENT_NAME, global_step=self.snapID)


SamplePreprosessor.py

In [ ]:
def preprocess(img):
    "scale image into the desired imgSize, transpose it for TF and normalize gray-values"

    # increase dataset size by applying random stretches to the images
    if AUGMENT_IMAGE:
        stretch = (random.random() - 0.5)  # -0.5 .. +0.5
        # random width, but at least 1
        wStretched = max(int(img.shape[1] * (1 + stretch)), 1)
        # stretch horizontally by factor 0.5 .. 1.5
        img = cv2.resize(img, (wStretched, img.shape[0]))

    # create target image and copy sample image into it
    (h, w) = img.shape
    fx = w / IMAGE_WIDTH
    fy = h / IMAGE_HEIGHT
    f = max(fx, fy)
    # scale according to f (result at least 1 and at most wt or ht)
    newSize = (max(min(IMAGE_WIDTH, int(w / f)), 1),
               max(min(IMAGE_HEIGHT, int(h / f)), 1))
    img = cv2.resize(img, newSize)
    target = np.ones([IMAGE_HEIGHT, IMAGE_WIDTH]) * 255
    target[0:newSize[1], 0:newSize[0]] = img

    # transpose for TF
    img = cv2.transpose(target)

    # normalize
    (m, s) = cv2.meanStdDev(img)
    m = m[0][0]
    s = s[0][0]
    img = img - m
    img = img / s if s > 0 else img

    return img


DataGenerator_BinaryFile.py

In [ ]:
class Sample:
    "a single sample from the dataset"

    def __init__(self, gtText, imageIdx, imageHeight, imageWidth, imageSize, imageStartPosition):
        self.gtText = gtText
        self.imageIdx = imageIdx
        self.imageHeight = imageHeight
        self.imageWidth = imageWidth
        self.imageSize = imageSize
        self.imageStartPosition = imageStartPosition


class Batch:
    "batch containing images and ground truth texts"

    def __init__(self, gtTexts, imgs):
        self.gtTexts = gtTexts
        self.imgs = np.stack(imgs, axis=0)


class DataGenerator:

    def __init__(self):
        self.binaryImageFile = open(BASE_IMAGES_FILE, "rb")
        self.currIdx = 0
        self.samples = []
        self.trainSamples = []
        self.validationSamples = []
        self.testSamples = []

    def LoadData(self, operationType):
        if not os.path.isfile(TRAINING_LABELS_FILE) \
                or not os.path.isfile(VALIDATION_LABELS_FILE) \
                or not os.path.isfile(TESTING_LABELS_FILE):
            self.createDataFiles()

        if operationType == OperationType.Training:
            self.loadDataFile(OperationType.Training)
            self.loadDataFile(OperationType.Validation)
        elif operationType == OperationType.Validation:
            self.loadDataFile(OperationType.Validation)
        elif operationType == OperationType.Testing:
            self.loadDataFile(OperationType.Testing)

    def createDataFiles(self):
        charsSet = set()
        wordsSet = set()

        f = open(BASE_LABELS_FILE, encoding="utf-8")
        for line in f:
            # read all samples ==> append line as is
            self.samples.append(line)

            if REGENERATE_CHARLIST_AND_CORPUS:
                # extract unique characters from text
                lineSplit = line.split(';')
                gtText = lineSplit[8]
                gtText = gtText[5:]
                wordsSet.add(gtText)
                charsSet = charsSet.union(set(list(gtText)))

        f.close()

        # create a text file that contains all the characters in the dataset
        # this list shall used to create the CTC model
        # There might be a problem if a previously saved model used larger data, consequently, not all
        # the characters in the previous model will be generated and therefore RNN creation will fail
        # note that a problem might arise when we try to open a saved model that was saved on a larger dataset
        # conseuqnelty some represented characters might be abscent and the new model will fail to load previous one
        # a solution for this problem is to use a static character set for the used dataset
        # also create the corpus data file for BeamSearch (if required)

        # DONT CREATE THEM UNLESS U R USING LARGER DATASET, ALREADY CREATED IN DIRECTORY
        if REGENERATE_CHARLIST_AND_CORPUS:
            localCharList = sorted(list(charsSet))
            open(fnCharList, 'w',
                 encoding="utf-8").write(str().join(localCharList))
            open(fnCorpus, 'w',
                 encoding="utf-8").write(str().join(sorted(list(wordsSet))))

        # first of all, make sure to randomly shuffle the main lables file
        # random.shuffle(self.samples)

        # split into training, validation, testing
        lenOfAllSamples = len(self.samples)
        lenOfTrainSamples = int(TRAINING_DATASET_SIZE * lenOfAllSamples)
        lenOfTrainingAndValidationSamples = lenOfAllSamples - lenOfTrainSamples
        lenOfValidationSamples = int(
            VALIDATION_DATASET_SPLIT_SIZE * lenOfTrainingAndValidationSamples)

        with open(TRAINING_LABELS_FILE, 'w', encoding="utf-8") as f:
            for item in self.samples[:lenOfTrainSamples]:
                f.write(item)

        with open(VALIDATION_LABELS_FILE, 'w', encoding="utf-8") as f:
            for item in self.samples[lenOfTrainSamples:lenOfTrainSamples + lenOfValidationSamples]:
                f.write(item)

        with open(TESTING_LABELS_FILE, 'w', encoding="utf-8") as f:
            for item in self.samples[lenOfTrainSamples + lenOfValidationSamples:]:
                f.write(item)

        self.samples = []

    def loadDataFile(self, operationType):
        if operationType == OperationType.Training:
            fileName = TRAINING_LABELS_FILE
        elif operationType == OperationType.Validation:
            fileName = VALIDATION_LABELS_FILE
        elif operationType == OperationType.Testing:
            fileName = TESTING_LABELS_FILE

        f = open(fileName, encoding="utf-8")
        for line in f:
            lineSplit = line.split(';')

            imgIdx = lineSplit[0]
            imgIdx = imgIdx[10:]

            imgStartPosition = lineSplit[1]
            imgStartPosition = int(imgStartPosition[15:])

            imgHeight = lineSplit[2]
            imgHeight = int(imgHeight[13:])
            imgWidth = lineSplit[3]
            imgWidth = int(imgWidth[12:])
            imgSize = imgHeight * imgWidth

            gtText = lineSplit[8]
            gtText = gtText[5:]
            #gtText = self.truncateLabel(' '.join(gtText), MAX_TEXT_LENGTH)

            # put sample into list
            if operationType == OperationType.Training:
                self.trainSamples.append(
                    Sample(gtText, imgIdx, imgHeight, imgWidth, imgSize, imgStartPosition))
            elif operationType == OperationType.Validation:
                self.validationSamples.append(
                    Sample(gtText, imgIdx, imgHeight, imgWidth, imgSize, imgStartPosition))
            elif operationType == OperationType.Testing:
                self.testSamples.append(
                    Sample(gtText, imgIdx, imgHeight, imgWidth, imgSize, imgStartPosition))

    def truncateLabel(self, text, maxTextLen):
        # ctc_loss can't compute loss if it cannot find a mapping between text label and input
        # labels. Repeat letters cost double because of the blank symbol needing to be inserted.
        # If a too-long label is provided, ctc_loss returns an infinite gradient
        cost = 0
        for i in range(len(text)):
            if i != 0 and text[i] == text[i - 1]:
                cost += 2
            else:
                cost += 1
            if cost > maxTextLen:
                return text[:i]
        return text

    def selectTrainingSet(self):
        "switch to randomly chosen subset of training set"
        self.currIdx = 0
        random.shuffle(self.trainSamples)
        self.samples = self.trainSamples[:TRAINING_SAMPLES_PER_EPOCH]

    def selectValidationSet(self):
        "switch to validation set"
        self.currIdx = 0
        random.shuffle(self.validationSamples)
        self.samples = self.validationSamples[:
                                              VALIDATIOIN_SAMPLES_PER_STEP]

    def selectTestSet(self):
        "switch to validation set"
        self.currIdx = 0
        random.shuffle(self.testSamples)
        self.samples = self.testSamples[:VALIDATIOIN_SAMPLES_PER_STEP]

    def getIteratorInfo(self):
        "current batch index and overall number of batches"
        return (self.currIdx // BATCH_SIZE + 1, len(self.samples) // BATCH_SIZE)

    def hasNext(self):
        "iterator"
        return self.currIdx + BATCH_SIZE <= len(self.samples)

    def getNext(self):
        "iterator"
        batchRange = range(self.currIdx, self.currIdx + BATCH_SIZE)
        gtTexts = [self.samples[i].gtText for i in batchRange]

        imgs = []
        for i in batchRange:
            try:
                self.binaryImageFile.seek(self.samples[i].imageStartPosition)
                img = np.frombuffer(self.binaryImageFile.read(
                    self.samples[i].imageSize), np.dtype('B'))
                img = img.reshape(
                    self.samples[i].imageHeight, self.samples[i].imageWidth)
                img = preprocess(img)
                # img = preprocess(img, IMAGE_WIDTH, IMAGE_HEIGHT, RESIZE_IMAGE,
                #                  CONVERT_IMAGE_TO_MONOCHROME, AUGMENT_IMAGE)
                imgs.append(img)
            except IOError as e:
                print("I/O error({0}): {1}".format(e.errno, e.strerror))
                pass
            except ValueError as e:
                print(e)
                pass
            except Error as e:
                print("Unexpected error:", sys.exc_info()[0])
                print("Value error({0}): {1}".format(e.errno, e.strerror))

                pass

        self.currIdx += BATCH_SIZE
        return Batch(gtTexts, imgs)



main.py

In [ ]:

startTime = datetime.now()
totalProcessingTime = 0


# we only need DataGenerator in training, validation, testing inorder to access the related datasets
if OPERATION_TYPE != OperationType.Infer:
    dataGenerator = DataGenerator()

def accumulateProcessingTime(paraTimeSnapshot):
    totalProcessingTime = time.time()
    #totalProcessingTime = totalProcessingTime + (time.time() - paraTimeSnapshot)

def train(paraModel):
    "train NN"
    epoch = 0  # number of training epochs since start
    bestCharErrorRate = float('inf')  # best valdiation character error rate
    noImprovementSince = 0  # number of epochs no improvement of character error rate occured

    auditString = get_initial_status_log()
    print(auditString)
    auditLog(auditString)

    continueLooping = True

    while continueLooping:
        print("Current Time =", datetime.now())

        epoch += 1
        print('Epoch:', epoch)

        dataGenerator.selectTrainingSet()

        while dataGenerator.hasNext():

            timeSnapshot = time.time()

            iterInfo = dataGenerator.getIteratorInfo()
            batch = dataGenerator.getNext()
            loss = paraModel.trainBatch(batch)

            # #stop execution after reaching a certain threashold
            # if (int(loss) == 1):
            #     noImprovementSince = MAXIMUM_NONIMPROVED_EPOCHS;

            print('Training Batch:', iterInfo[0],
                  '/', iterInfo[1], 'Loss:', loss)

            accumulateProcessingTime(timeSnapshot)

        # validate
        charErrorRate, charSuccessRate, wordsSuccessRate = validate(
            paraModel, OperationType.Validation)
        auditString = "Epoch Number %d." % epoch + "\n"

        # if best validation accuracy so far, save model parameters
        if charErrorRate < bestCharErrorRate:
            auditString = auditString + 'Character error rate improved, saving model'
            paraModel.save()
            bestCharErrorRate = charErrorRate
            noImprovementSince = 0
        else:
            auditString = auditString + "Character error rate not improved\n"
            noImprovementSince += 1

        # stop training if no more improvement in the last x epochs
        if noImprovementSince >= MAXIMUM_NONIMPROVED_EPOCHS:
            auditString = auditString + \
                "No more improvement since %d epochs." % MAXIMUM_NONIMPROVED_EPOCHS + "\n"

            # gracefull termination
            continueLooping = False

        # Model did not finish, print log and save it
        auditString = auditString + \
            get_execution_log(charSuccessRate, wordsSuccessRate)
        print(auditString)
        auditLog(auditString)


def validate(paraModel, paraOperationType):
    if paraOperationType == OperationType.Validation:
        dataGenerator.selectValidationSet()

    elif paraOperationType == OperationType.Testing:
        dataGenerator.selectTestSet()

    numCharErr = 0
    numCharTotal = 0
    numWordOK = 0
    numWordTotal = 0
    timeSnapshot = 0.0

    while dataGenerator.hasNext():
        timeSnapshot = time.time()

        iterInfo = dataGenerator.getIteratorInfo()
        print('Validating Batch:', iterInfo[0], '/', iterInfo[1])
        batch = dataGenerator.getNext()
        (recognized, _) = paraModel.inferBatch(batch)

        accumulateProcessingTime(timeSnapshot)

        # print('Ground truth -> Recognized')
        for i in range(len(recognized)):
            numWordTotal += 1
            numCharTotal += len(batch.gtTexts[i])

            numWordOK += 1 if batch.gtTexts[i] == recognized[i] else 0

            dist = editdistance.eval(recognized[i], batch.gtTexts[i])
            numCharErr += dist

            # remove remark to see each success and error values
            #print('[OK]' if dist==0 else '[ERR:%d]' % dist,'"' + batch.gtTexts[i] + '"', '->', '"' + recognized[i] + '"')

    # print validation result
    charErrorRate = numCharErr / numCharTotal
    charSuccessRate = 1 - (numCharErr / numCharTotal)
    wordsSuccessRate = numWordOK / numWordTotal

    # print and save validation result, this includes post epoch operation as well as when
    # running standalone testing or validation processes

    return charErrorRate, charSuccessRate, wordsSuccessRate


def inferSingleImage(paraModel, paraFnImg):
    "recognize text in image provided by file path"
    img = cv2.imread(paraFnImg, cv2.IMREAD_GRAYSCALE)
    img = preprocess(img)
    # img = preprocess(img, IMAGE_WIDTH,
    #                  IMAGE_HEIGHT, True, False, False)

    batch = Batch(None, [img])
    #(recognized, probability) = model.inferBatch(batch)
    (recognized, probability) = paraModel.inferBatch(batch, True)
    print('Recognized:', '"' + recognized[0] + '"')
    print('Probability:', probability[0])


def get_initial_status_log():
    auditString = "____________________________________________________________" + "\n"
    auditString = auditString + "Experiment Name: " + EXPERIMENT_NAME + "\n"
    auditString = auditString + "Base File Name: " + BASE_FILENAME + "\n"
    auditString = auditString + 'Start Execution Time :' + \
        startTime.strftime("%m/%d/%Y, %H:%M:%S") + "\n"
    auditString = auditString + "Training set size: " + \
        str(len(dataGenerator.trainSamples)) + "\n"
    auditString = auditString + "Validation set size: " + \
        str(len(dataGenerator.validationSamples)) + "\n"
    auditString = auditString + "Training Samples per epoch: " + \
        str(TRAINING_SAMPLES_PER_EPOCH) + "\n"
    auditString = auditString + "Validation Samples per step: " + \
        str(VALIDATIOIN_SAMPLES_PER_STEP) + "\n"
    auditString = auditString + "Batch size: " + str(BATCH_SIZE) + "\n"
    auditString = auditString + "TRAINING_SAMPLES_PER_EPOCH: " + \
        str(TRAINING_SAMPLES_PER_EPOCH) + "\n"
    auditString = auditString + "BATCH_SIZE: " + str(BATCH_SIZE) + "\n"
    auditString = auditString + "VALIDATIOIN_SAMPLES_PER_STEP: " + \
        str(VALIDATIOIN_SAMPLES_PER_STEP) + "\n"
    auditString = auditString + "TRAINING_DATASET_SIZE: " + \
        str(TRAINING_DATASET_SIZE) + "\n"
    auditString = auditString + "VALIDATION_DATASET_SPLIT_SIZE: " + \
        str(VALIDATION_DATASET_SPLIT_SIZE) + "\n"
    auditString = auditString + "IMAGE_WIDTH: " + \
        str(IMAGE_WIDTH) + "\n"
    auditString = auditString + "IMAGE_HEIGHT: " + \
        str(IMAGE_HEIGHT) + "\n"
    auditString = auditString + "MAX_TEXT_LENGTH: " + \
        str(MAX_TEXT_LENGTH) + "\n"
    auditString = auditString + "RESIZE_IMAGE: " + \
        str(RESIZE_IMAGE) + "\n"
    auditString = auditString + "CONVERT_IMAGE_TO_MONOCHROME: " + \
        str(CONVERT_IMAGE_TO_MONOCHROME) + "\n"
    auditString = auditString + "MONOCHROME_BINARY_THRESHOLD: " + \
        str(MONOCHROME_BINARY_THRESHOLD) + "\n"
    auditString = auditString + "AUGMENT_IMAGE: " + \
        str(AUGMENT_IMAGE) + "\n\n"

    return auditString


def get_execution_log(paraCharSuccessRate, paraWordsSuccessRate):
    auditString = "Start Execution Time : " + \
        startTime.strftime("%m/%d/%Y, %H:%M:%S") + "\n"
    auditString = auditString + "End Execution Time  :" + \
        datetime.now().strftime("%m/%d/%Y, %H:%M:%S") + "\n"
    auditString = auditString + "Accumulated Processing Time : " + \
        str(totalProcessingTime / 60) + " minutes" + "\n"
    auditString = auditString + "Characters Success Rate: " + \
        str(paraCharSuccessRate * 100.0) + "%\n"
    auditString = auditString + "Words Success Rate: " + \
        str(paraWordsSuccessRate * 100.0) + "%\n\n"

    return auditString



def main():

    if OPERATION_TYPE != OperationType.Infer:
        dataGenerator.LoadData(OPERATION_TYPE)

    if OPERATION_TYPE == OperationType.Training:
        auditString = "EXPERIMENT_NAME: " + EXPERIMENT_NAME + "\n"
        auditString = auditString + "Training Using Dataset: " + \
            str(OPERATION_TYPE) + "\n"
        print(auditString)
        auditLog(auditString)

        model = Model(DECODER_TYPE, mustRestore=False, dump=False)
        train(model)
    elif OPERATION_TYPE == OperationType.Validation or OPERATION_TYPE == OperationType.Testing:
        auditString = "EXPERIMENT_NAME: " + EXPERIMENT_NAME + "\n"
        auditString = auditString + "Validation/Tesing Using Dataset: " + \
            str(OPERATION_TYPE) + "\n"
        print(auditString)

        model = Model(DECODER_TYPE, mustRestore=True, dump=False)
        charErrorRate, charSuccessRate, wordsSuccessRate = validate(
            model, OPERATION_TYPE)

        auditString = auditString + \
            get_execution_log(charSuccessRate, wordsSuccessRate) + "\n"
        print(auditString)

        auditLog(auditString)

    elif OPERATION_TYPE == OperationType.Infer:  # infer text on test image
        print(open(fnResult).read())
        tf.reset_default_graph()
        #model = Model(open(fnCharList, encoding="utf-8").read(), decoderType, mustRestore=True, dump=args.dump)
        model = Model(DECODER_TYPE, mustRestore=True, dump=False)
        inferSingleImage(model, fnInfer)


if __name__ == '__main__':    
    main()
